In [1]:
import imageio
import numpy as np

# Dev stuff

In [2]:
samples = {
    "small" : "tests/small.bmp",
    "small-alpha" : "tests/small-alpha.bmp",
    "shadow" : "tests/proprietary/warhammer_map_1_1_shadow.bmp",
    "vortex" : "tests/proprietary/wh2_main_great_vortex_map_6.bmp"
}

# Input
Write your inputs here:

In [3]:
source = samples["small-alpha"]
relateDiagonals = True # Default is True
print(source)

tests/small-alpha.bmp


# Process
(Output is after this section)

In [4]:
image = imageio.imread(source)
print(image.shape)
print(image.meta)

adjacencies = dict()
nbRows = image.shape[0]
nbColumns = image.shape[1]
maxRow = nbRows - 1
maxColumn = nbColumns -1
print(nbRows, nbColumns, maxRow, maxColumn)

(8, 8, 4)
Dict([('dpi', (96, 96)), ('compression', 3)])
8 8 7 7


In [5]:
# Print pixel data for debug
print("\t".join(["row", "col", "r", "g", "b", "a"]))
for row in range(nbRows):
    for column in range(nbColumns):
        print(row, column, "\t".join(map(str, image[row, column])), sep="\t")
        
R_INDEX = 0
G_INDEX = 1
B_INDEX = 2
A_INDEX = 3

row	col	r	g	b	a
0	0	255	0	0	255
0	1	255	0	0	255
0	2	255	0	0	255
0	3	255	0	0	255
0	4	255	0	0	255
0	5	255	0	255	255
0	6	255	0	255	255
0	7	255	255	0	255
1	0	255	0	0	255
1	1	255	0	0	255
1	2	255	0	0	255
1	3	255	0	0	255
1	4	0	0	255	255
1	5	255	0	255	255
1	6	255	0	255	255
1	7	255	0	255	255
2	0	0	255	255	255
2	1	0	255	255	255
2	2	255	0	0	255
2	3	255	0	0	255
2	4	0	0	255	255
2	5	0	0	255	255
2	6	0	0	255	255
2	7	255	0	255	255
3	0	0	255	255	255
3	1	0	255	255	255
3	2	255	0	0	255
3	3	255	0	0	255
3	4	0	0	255	255
3	5	0	0	255	255
3	6	0	0	255	255
3	7	255	0	255	255
4	0	0	255	255	255
4	1	0	255	255	255
4	2	0	255	255	255
4	3	0	255	255	255
4	4	0	255	0	255
4	5	0	255	0	255
4	6	0	255	0	255
4	7	255	0	255	255
5	0	0	255	255	255
5	1	0	255	255	255
5	2	0	255	255	255
5	3	0	255	255	255
5	4	0	255	0	255
5	5	0	255	0	255
5	6	0	255	0	255
5	7	255	0	255	255
6	0	0	255	255	255
6	1	0	255	255	255
6	2	0	255	255	255
6	3	0	255	255	255
6	4	0	255	0	255
6	5	0	255	0	255
6	6	0	255	0	255
6	7	255	0	255	255
7	0	0	255	255	255
7	1	0	255	255	25

In [6]:
def process_pixel(pixelRow, pixelColumn):
    pixelColour = tuple(image[pixelRow, pixelColumn].tolist())
    process_neighbour(pixelColour, pixelRow, pixelColumn, 1, -1)
    process_neighbour(pixelColour, pixelRow, pixelColumn, 1, 1)
    process_neighbour(pixelColour, pixelRow, pixelColumn, -1, -1)
    process_neighbour(pixelColour, pixelRow, pixelColumn, -1, 1)
    if relateDiagonals:
        process_neighbour(pixelColour, pixelRow, pixelColumn, 1, 0)
        process_neighbour(pixelColour, pixelRow, pixelColumn, 0, 1)
        process_neighbour(pixelColour, pixelRow, pixelColumn, 0, -1)
        process_neighbour(pixelColour, pixelRow, pixelColumn, -1, 0)

        
def process_neighbour(pixelColour, pixelRow, pixelColumn, rowOffset, columnOffset):
    neighRow = pixelRow + rowOffset
    neighColumn = pixelColumn
    if not valid_row_column(neighRow, neighColumn): 
        return
    neighColour = tuple(image[neighRow, neighColumn].tolist())
    if same_colours(pixelColour, neighColour): 
        return
    adjacencies.setdefault(pixelColour, set()).add(neighColour)
    
    
def valid_row_column(row, column):
    return (0 <= row
           and 0 <= column
           and row <= maxRow
           and column <= maxColumn)

def same_colours(a, b):
    if len(a) != len(b):
        raise TypeError("Colours from the same image should have the same number of channels.")
    for i in range(len(a)):
        if a[i] != b[i]:
            return False
    return True


for row in range(nbRows):
    for column in range(nbColumns):
        process_pixel(row, column)

print(adjacencies)

{(255, 0, 0, 255): {(0, 0, 255, 255), (0, 255, 255, 255)}, (255, 255, 0, 255): {(0, 255, 255, 255), (0, 255, 0, 255), (255, 0, 255, 255)}, (0, 0, 255, 255): {(255, 0, 255, 255), (0, 255, 0, 255), (255, 0, 0, 255)}, (255, 0, 255, 255): {(0, 0, 255, 255), (255, 255, 0, 255)}, (0, 255, 255, 255): {(255, 255, 0, 255), (255, 0, 0, 255)}, (0, 255, 0, 255): {(0, 0, 255, 255), (255, 255, 0, 255)}}
